In [37]:
import cvxpy as cp
from cvxpy.atoms.affine.bmat import bmat
import numpy as np
import matplotlib.pyplot as plt

def make_signal(n_spikes, amplitude_noise): 
    spikes = (0.5 + np.arange(n_spikes)) / n_spikes
    phases = np.random.uniform(size=(n_spikes,))
    amplitudes = np.random.normal(size=(n_spikes,))
    return spikes, amplitudes * np.exp(2 * np.pi * 1j * phases)

def plot_signal(signal):
    spikes, vals = signal
    plt.plot(spikes, np.real(vals))
    plt.plot([0, 1], [0, 0], "r--")
    
def measure_signal(signal, cutoff):
    spikes, vals = signal
    measure_freq = lambda k: np.dot(vals, np.exp(2 * np.pi * 1j * k * spikes))
    return np.array([measure_freq(k) for k in np.arange(-cutoff, cutoff+1)]).reshape((-1, 1))

def evaluate_dual(X, fourier_coeffs):
    f_c = (fourier_coeffs.shape[0] - 1) // 2
    n = X.shape[0]
    idft = np.array([np.exp(-2 * np.pi * 1j * k * X[i]) for i in range(n) for j in range(-f_c, f_c+1)])



In [29]:
n_spikes = 20
f_c = 3*n
d = 2*f_c + 1

SIGNAL = make_signal(n_spikes, 1)
Y = measure_signal(SIGNAL, f_c)

In [33]:
Q = cp.Variable((d, d), hermitian=True) # is_hermitian makes Q complex automatically
c = cp.Variable((d, 1), complex=True)


constraints = [bmat([[Q, c], [c.H, np.eye(1)]]) >> 0] + \
    [cp.trace(np.eye(d, k=j).T @ Q) == (1 if j==0 else 0) for j in range(d)]

objective = cp.Maximize(cp.real(c.H @ Y))


In [34]:
prob = cp.Problem(objective, constraints)
prob.solve()
c_opt = c.value

15.898641005255497